# CSV 데이터 로드

- `tf.data.Dataset`를 이용한 CSV 데이터 로드
- 타이타닉 데이터 로드 연습

---

### 설정

- `tf.keras.utils.get_file`는 두 번째 파라미터인 URL로부터 첫 번째 인자인 파일을 가져온다.
- 파일이 저장되는 기본 경로는 Mac OS 기준 /Users/\<user name\>/.keras/datasets 이다.

In [1]:
import functools

import numpy as np
import tensorflow as tf

In [2]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

24576/13049 [========================================================] - 0s 0us/step


In [45]:
print(train_file_path, test_file_path)

/Users/mazdah/.keras/datasets/train.csv /Users/mazdah/.keras/datasets/eval.csv


In [47]:
# 화면에 출력될 값을 읽기 쉽도록 조정하는 함수
np.set_printoptions(precision=3, suppress=True)

---

### 데이터 로드

- 큰 파일 세트로 확장하거나 tf.datad와 통합되어야 하는 경우 `tf.data.experimental.make_csv_dataset` 함수 사용
- 예측해야 할 값과 관련된 변수

In [8]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

- 파일에서 CSV 데이터를 읽고 데이터세트를 작성
- `tf.data.experimental.make_csv_dataset`은 CSV 파일로부터 Dataset 객체를 만든다.

> 참조 URL : [tf.data.experimental.make_csv_dataset](https://www.tensorflow.org/api_docs/python/tf/data/experimental/make_csv_dataset)

In [49]:
# Python 기본 : **kwargs는 (키워드=값) 형태의 파라미터를 다수 받을 수 있다는 의미이다.
#              *args는 다수의 파라미터를 받을 수 있다는 의미이며 튜플의 형태이다.
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,    # label_name 파라미터로 지정된 컬럼은 별도의 Tensor로 분리된다.
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [50]:
print(raw_train_data)
print(raw_test_data)

<PrefetchDataset shapes: (OrderedDict([(sex, (None,)), (age, (None,)), (n_siblings_spouses, (None,)), (parch, (None,)), (fare, (None,)), (class, (None,)), (deck, (None,)), (embark_town, (None,)), (alone, (None,))]), (None,)), types: (OrderedDict([(sex, tf.string), (age, tf.float32), (n_siblings_spouses, tf.int32), (parch, tf.int32), (fare, tf.float32), (class, tf.string), (deck, tf.string), (embark_town, tf.string), (alone, tf.string)]), tf.int32)>
<PrefetchDataset shapes: (OrderedDict([(sex, (None,)), (age, (None,)), (n_siblings_spouses, (None,)), (parch, (None,)), (fare, (None,)), (class, (None,)), (deck, (None,)), (embark_town, (None,)), (alone, (None,))]), (None,)), types: (OrderedDict([(sex, tf.string), (age, tf.float32), (n_siblings_spouses, tf.int32), (parch, tf.int32), (fare, tf.float32), (class, tf.string), (deck, tf.string), (embark_town, tf.string), (alone, tf.string)]), tf.int32)>


In [51]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

- 데이터세트의 각 항목은 하나의 배치이며, (다수의 예제, 다수의 레이블)의 튜플로 표현됨 
- 예제의 데이터는 행 기반 텐서가 아닌 열 기반 텐서로 구성
- 각 데이터는 배치 크기(이 경우 5)만큼 많은 요소가 있음

In [13]:
show_batch(raw_train_data)

sex                 : [b'female' b'male' b'female' b'male' b'male']
age                 : [32. 28. 44. 31. 19.]
n_siblings_spouses  : [1 0 0 0 0]
parch               : [1 0 0 0 0]
fare                : [15.5   30.5   27.721  7.775 10.5  ]
class               : [b'Third' b'First' b'First' b'Third' b'Second']
deck                : [b'unknown' b'C' b'B' b'unknown' b'unknown']
embark_town         : [b'Queenstown' b'Southampton' b'Cherbourg' b'Southampton' b'Southampton']
alone               : [b'n' b'y' b'y' b'y' b'y']


2022-04-11 10:22:57.184059: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


- 기본적으로 CSV의 열 이름이 지정되나 첫 번째 줄에 열 이름이 없는 경우 열 이름을 문자열 목록으로 `make_csv_dataset` 함수의 `column_names` 인수로 전달하여 처리

In [14]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'male' b'male' b'female' b'male' b'male']
age                 : [ 0.83 26.   22.   43.   20.  ]
n_siblings_spouses  : [0 2 0 0 1]
parch               : [2 0 1 0 1]
fare                : [29.     8.663 55.     8.05  15.742]
class               : [b'Second' b'Third' b'First' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'E' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton' b'Cherbourg']
alone               : [b'n' b'n' b'n' b'y' b'n']


- 데이터세트에서 일부 열을 생략해야 하는 경우, 사용하려는 열의 목록만 작성하고 생성자의 (선택적) `select_columns` 인수로 전달
- `make_csv_datase` 함수는 `select_columns`로 전달받은 컬럼명들에 해당하는 컬럼들만을 뽑아서 dataset을 만든다 

In [15]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [28. 28. 28. 36. 63.]
n_siblings_spouses  : [0 0 0 1 1]
class               : [b'Third' b'Third' b'Third' b'Second' b'First']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'D']
alone               : [b'y' b'y' b'y' b'n' b'n']


---

### 데이터 전처리
- 일반적으로 데이터를 모델에 공급하기 전에 혼합 유형에서 고정 길이 벡터로 변환
- `tf.feature_column` : TensorFlow의 일반적인 입력 변환을 설명하기 위한 내장 시스템
- 모델 내에서 전처리를 수행할 때의 주요 이점은 모델을 내보낼 때 전처리가 포함된다는 것. 이렇게 하면 원시 데이터를 모델로 직접 전달할 수 있음.

#### 연속 데이터
- 데이터가 이미 적절한 숫자 형식인 경우, 데이터를 모델로 전달하기 전에 벡터로 묶음

In [16]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [64. 32. 25. 27.  7.]
n_siblings_spouses  : [1. 1. 0. 0. 4.]
parch               : [4. 1. 0. 0. 1.]
fare                : [263.     15.5     0.      7.896  39.688]


In [17]:
example_batch, labels_batch = next(iter(temp_dataset))

- 모든 열을 묶는 간단한 함수

In [18]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

- 이 함수를 데이터세트의 각 요소에 적용

In [19]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

[[28.     0.     0.     7.729]
 [52.     1.     1.    93.5  ]
 [19.     0.     0.    10.5  ]
 [42.     1.     0.    52.   ]
 [32.     0.     0.    56.496]]

[0 1 0 0 1]


- 혼합 데이터 유형이 있는 경우, 해당 단순 숫자 필드를 분리할 수 있음 
- `tf.feature_column` API로 처리할 수 있지만, 약간의 오버헤드가 발생하며 실제로 필요하지 않으면 피해야 함

In [20]:
show_batch(raw_train_data)

sex                 : [b'male' b'female' b'female' b'male' b'male']
age                 : [35. 28. 28. 28. 24.]
n_siblings_spouses  : [0 1 0 0 0]
parch               : [0 0 0 0 0]
fare                : [ 26.288 146.521   7.75    7.229  79.2  ]
class               : [b'First' b'First' b'Third' b'Third' b'First']
deck                : [b'E' b'B' b'unknown' b'unknown' b'B']
embark_town         : [b'Southampton' b'Cherbourg' b'Queenstown' b'Cherbourg' b'Cherbourg']
alone               : [b'y' b'n' b'y' b'y' b'y']


In [21]:
example_batch, labels_batch = next(iter(temp_dataset))

- 숫자 특성 목록을 선택하고 단일 열로 묶는 보다 일반적인 전처리기를 정의

In [22]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [23]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [24]:
show_batch(packed_train_data)

sex                 : [b'male' b'female' b'female' b'male' b'male']
class               : [b'Third' b'First' b'First' b'Third' b'Third']
deck                : [b'unknown' b'B' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'y' b'n' b'y' b'n' b'n']
numeric             : [[ 28.      0.      0.     56.496]
 [ 14.      1.      2.    120.   ]
 [ 22.      0.      0.    151.55 ]
 [ 28.      1.      0.     15.85 ]
 [  1.      1.      2.     20.575]]


In [25]:
example_batch, labels_batch = next(iter(packed_train_data))

---

#### 데이터 정규화
- 연속 데이터는 항상 정규화 되어야 한다!

In [26]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [27]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [28]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

- 숫자 열 생성, `tf.feature_columns.numeric_column` API는 각 배치에서 실행될 `normalizer_fn` 인수를 허용
- `functools.partial`를 사용하여 MEAN 및 STD를 노멀라이저 fn에 바인딩한다.

In [29]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x118005820>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

- 모델을 훈련할 때 이 특성 열을 포함하여 이 숫자 데이터 블록을 선택하고 중앙에 배치

In [30]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[28.   ,  8.   ,  2.   , 69.55 ],
       [27.   ,  1.   ,  0.   , 13.858],
       [ 3.   ,  1.   ,  1.   , 18.75 ],
       [28.   ,  0.   ,  0.   ,  7.229],
       [45.   ,  0.   ,  0.   , 26.55 ]], dtype=float32)>

In [31]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-0.13 ,  6.476,  2.043,  0.644],
       [-0.21 ,  0.395, -0.479, -0.376],
       [-2.128,  0.395,  0.782, -0.286],
       [-0.13 , -0.474, -0.479, -0.497],
       [ 1.228, -0.474, -0.479, -0.144]], dtype=float32)

- 평균 기반 정규화를 위해서는 각 열의 평균을 미리 알아야 한다.

---

#### 범주형 데이터
- `tf.feature_column` API를 사용하여 각 범주 열에 대해 `tf.feature_column.indicator_column`을 가진 모음을 작성

In [32]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [33]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [34]:
# See what you just created.
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [35]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


- 이것은 나중에 모델을 빌드할 때 데이터 처리 입력의 일부가 된다.

---

#### 결합된 전처리 레이어
- 두 개의 특성 열 모음을 추가하고 `tf.keras.layers.DenseFeatures`에 전달하여 두 입력 유형을 추출하고 전처리할 입력 레이어를 만든다.

In [36]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [37]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 0.     1.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.    -0.13   6.476
  2.043  0.644  0.     1.   ]


---

### 모델 빌드하기
- `preprocessing_layer`를 사용하여 `tf.keras.Sequential`를 빌드

In [38]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1),
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

---

### 훈련, 평가 및 예측하기

In [39]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [40]:
model.fit(train_data, epochs=20)

Epoch 1/20
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.


2022-04-12 10:43:53.153963: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


126/126 [==============================] - 3s 15ms/step - loss: 0.5090 - accuracy: 0.7496
Epoch 2/20
126/126 [==============================] - 1s 10ms/step - loss: 0.4149 - accuracy: 0.8150
Epoch 3/20
126/126 [==============================] - 1s 10ms/step - loss: 0.4010 - accuracy: 0.8150
Epoch 4/20
126/126 [==============================] - 1s 10ms/step - loss: 0.3899 - accuracy: 0.8357
Epoch 5/20
126/126 [==============================] - 1s 10ms/step - loss: 0.3810 - accuracy: 0.8437
Epoch 6/20
126/126 [==============================] - 1s 11ms/step - loss: 0.3684 - accuracy: 0.8437
Epoch 7/20
126/126 [==============================] - 1s 10ms/step - loss: 0.3652 - accuracy: 0.8421
Epoch 8/20
126/126 [==============================] - 1s 11ms/step - loss: 0.3569 - accuracy: 0.8485
Epoch 9/20
126/126 [==============================] - 1s 10ms/step - loss: 0.3467 - accuracy: 0.8437
Epoch 10/20
126/126 [==============================] - 1s 11ms/step - loss: 0.3493 - accuracy: 0.8501


- test_data 세트에서 정확성을 확인

In [41]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

Consider rewriting this model with the Functional API.
     13/Unknown - 0s 9ms/step - loss: 0.4892 - accuracy: 0.8154 

2022-04-12 10:44:50.637899: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


53/53 [==============================] - 1s 11ms/step - loss: 0.4432 - accuracy: 0.8561


Test Loss 0.4431551396846771, Test Accuracy 0.8560606241226196


- `tf.keras.Model.predict`를 사용하여 배치 또는 배치 데이터세트에서 레이블을 유추

In [42]:
predictions = model.predict(test_data)

# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  prediction = tf.sigmoid(prediction).numpy()
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))

Consider rewriting this model with the Functional API.


2022-04-12 10:45:37.936242: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Predicted survival: 34.75%  | Actual outcome:  DIED
Predicted survival: 83.02%  | Actual outcome:  DIED
Predicted survival: 70.49%  | Actual outcome:  DIED
Predicted survival: 13.66%  | Actual outcome:  SURVIVED
Predicted survival: 78.98%  | Actual outcome:  SURVIVED
